# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import os
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();


df_grafos_propios = pd.read_csv("instancias/grafo-propio/indice.csv");



La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [4]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    display(metodo)
    print(instancia)
    
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../TSP", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    res = process.stdout.read().split(" ")
    tam = res[0] #tamaño del ciclo
    pyc =res[1].split("\n")
    peso = pyc[0] #peso del ciclo
    ciclo = pyc[1] #ciclo

    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, tam, peso, ciclo]; #devuelvo un par de cosas extra por si las queremos usar

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [5]:
experimentos = [];
#algoritmos = ["INS", "AGM", "GA",  "TABU-1", "TABU-2"]; asi seria finalizado
algoritmos = ["T_TABU"]#["ins"]; #de prueba para ver que funciona

### Experimento 1
Correr Fuerza Bruta en las primeras 30 instancias de densidad-alta y densidad-baja.

In [6]:

for n in range(0, df_grafos_propios.shape[0]):
    fila_n = df_grafos_propios.iloc[n];
    for algo in algoritmos:
        experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);
 

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [8]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "res"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        algoritmo = experimento[4]
        tiempos.append((correr_experimento(experimento[3], experimento[4]))[0]);
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 1/100'

'T_TABU'

'3 3\n1 2 97\n1 3 13\n2 3 28\n'

TypeError: exceptions must derive from BaseException